<a href="https://colab.research.google.com/github/NicoMontoya/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Have explored the data a little bit.  Now I will form a strategy!

first: 
- analyze top 100.  Breakdown of lefties vs righties in the latest ranking available.
- determine if difference from general population is statistically significant.
- compare to years past

second:
- analyze matches within each year of top 100 vs top 100 players.
- determine if lefties win more or less

Third/ bonus:
- analyze matches based on surface.
- determine if lefties have any advantage on a particular surface

Fourth
- do the exact same analysis for women

In [0]:
## performa classic imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from scipy import stats

In [0]:
## import player data
players_columns = ['player_id', 'first_name', 'last_name', 'hand', 'birth_date', 'country_code']
players = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv', names=players_columns, encoding = 'ISO-8859-1')

In [0]:
## import ranking data for 2018
rankings_columns = ['ranking_date', 'ranking', 'player_id', 'ranking_points']
current_ranking = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_current.csv', names=rankings_columns)

In [0]:
## merge players data with current rankings data
current_ranking = pd.merge(players, current_ranking, on='player_id')

In [0]:
current_ranking['ranking_date'] = pd.to_datetime(current_ranking['ranking_date'], format='%Y%m%d')

In [0]:
year_end_ranking2018 = current_ranking[current_ranking['ranking_date'] == '2018-09-24'].sort_values(by='ranking')

In [0]:
year_end_ranking2018 = year_end_ranking2018.head(100)

In [0]:
year_end_ranking2018 = year_end_ranking2018.replace({'R':0, 'L':1})

In [373]:
year_end_ranking2018.head()

,player_id,first_name,last_name,hand,birth_date,country_code,ranking_date,ranking,ranking_points
3735,104745,Rafael,Nadal,1,19860603.0,ESP,2018-09-24,1,8760
617,103819,Roger,Federer,0,19810808.0,SUI,2018-09-24,2,6900
4752,104925,Novak,Djokovic,0,19870522.0,SRB,2018-09-24,3,6445
7447,105223,Juan Martin,Del Potro,0,19880923.0,ARG,2018-09-24,4,5980
32,100644,Alexander,Zverev,0,19970420.0,GER,2018-09-24,5,4890


In [374]:
# import ranking data for 2010's
ranking2010s = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_10s.csv', names=rankings_columns)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
ranking2010s['ranking_date'] = pd.to_datetime(ranking2010s['ranking_date'], format='%Y%m%d')

In [0]:
## merge ranking2010s with players data
ranking2010s = pd.merge(players, ranking2010s, on='player_id')
ranking2010s = ranking2010s.replace({'R':0, 'L':1})

In [0]:
year_end_ranking2017 = ranking2010s[ranking2010s['ranking_date'] == '2017-12-25'].sort_values(by='ranking')

In [0]:
year_end_ranking2017 = year_end_ranking2017.head(100)

In [0]:
year_end_ranking2016 = ranking2010s[ranking2010s['ranking_date'] == '2016-12-26'].sort_values(by='ranking')

In [0]:
year_end_ranking2016 = year_end_ranking2016.head(100)

In [0]:
year_end_ranking2015 = ranking2010s[ranking2010s['ranking_date'] == '2015-12-28'].sort_values(by='ranking')

In [0]:
year_end_ranking2015 = year_end_ranking2015.head(100)

In [0]:
year_end_ranking2014 = ranking2010s[ranking2010s['ranking_date'] == '2014-12-29'].sort_values(by='ranking')
year_end_ranking2014 = year_end_ranking2014.head(100)

In [0]:
year_end_ranking2013 = ranking2010s[ranking2010s['ranking_date'] == '2013-12-30'].sort_values(by='ranking')
year_end_ranking2013 = year_end_ranking2013.head(100)

In [0]:
year_end_ranking2012 = ranking2010s[ranking2010s['ranking_date'] == '2012-12-31'].sort_values(by='ranking')
year_end_ranking2012 = year_end_ranking2012.head(100)

In [0]:
year_end_ranking2011 = ranking2010s[ranking2010s['ranking_date'] == '2011-12-26'].sort_values(by='ranking')
year_end_ranking2011 = year_end_ranking2011.head(100)

In [0]:
year_end_ranking2010 = ranking2010s[ranking2010s['ranking_date'] == '2010-12-27'].sort_values(by='ranking')
year_end_ranking2010 = year_end_ranking2010.head(100)

Let's analyze the top 100 rankings for the last 9 years for men

In [388]:
year_end_ranking2018['hand'].value_counts()

0    84
1    14
U     2
Name: hand, dtype: int64

In [389]:
year_end_ranking2017['hand'].value_counts()

0    84
1    16
Name: hand, dtype: int64

In [390]:
year_end_ranking2016['hand'].value_counts()

0    82
1    18
Name: hand, dtype: int64

In [391]:
year_end_ranking2015['hand'].value_counts()

0    87
1    13
Name: hand, dtype: int64

In [392]:
year_end_ranking2014['hand'].value_counts()

0    86
1    14
Name: hand, dtype: int64

In [393]:
year_end_ranking2013['hand'].value_counts()

0    86
1    14
Name: hand, dtype: int64

In [394]:
year_end_ranking2012['hand'].value_counts()

0    87
1    13
Name: hand, dtype: int64

In [395]:
year_end_ranking2011['hand'].value_counts()

0    84
1    16
Name: hand, dtype: int64

In [396]:
year_end_ranking2010['hand'].value_counts()

0    88
1    12
Name: hand, dtype: int64

Let's bring in the women

In [0]:
#import women players info
wtaplayers = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_players.csv', names=players_columns, encoding='ISO-8859-1')


In [0]:
#import recent rankings
wtarankings_columns = ['ranking_date', 'ranking', 'player_id', 'ranking_points', 'tours']
wtacurrent_ranking = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_rankings_current.csv', names=wtarankings_columns)

In [0]:
wtacurrent_ranking['ranking_date'] = pd.to_datetime(wtacurrent_ranking['ranking_date'], format='%Y%m%d')

In [0]:
wtacurrent_ranking = pd.merge(wtaplayers, wtacurrent_ranking, on='player_id')

In [0]:
wtayear_end_ranking2018 = wtacurrent_ranking[wtacurrent_ranking['ranking_date'] == '2018-12-24'].sort_values(by='ranking').head(100)

In [0]:
# import rankings for 2010s womens
wtaranking2010s = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_rankings_10s.csv', names = wtarankings_columns)

In [0]:
wtaranking2010s = pd.merge(wtaplayers, wtaranking2010s, on='player_id')

In [0]:
wtaranking2010s['ranking_date'] = pd.to_datetime(wtaranking2010s['ranking_date'], format='%Y%m%d')

In [0]:
wtayear_end_ranking2017 = wtaranking2010s[wtaranking2010s['ranking_date']=='2017-11-20'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2016 = wtaranking2010s[wtaranking2010s['ranking_date']=='2016-12-26'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2015 = wtaranking2010s[wtaranking2010s['ranking_date']=='2015-12-28'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2014 = wtaranking2010s[wtaranking2010s['ranking_date']=='2014-12-29'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2013 = wtaranking2010s[wtaranking2010s['ranking_date']=='2013-12-30'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2012 = wtaranking2010s[wtaranking2010s['ranking_date']=='2012-12-31'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2011 = wtaranking2010s[wtaranking2010s['ranking_date']=='2011-12-26'].sort_values(by='ranking').head(100)

In [0]:
wtayear_end_ranking2010 = wtaranking2010s[wtaranking2010s['ranking_date']=='2010-12-27'].sort_values(by='ranking').head(100)

In [433]:
wtayear_end_ranking2018['hand'].value_counts()

R    87
L     8
U     5
Name: hand, dtype: int64

In [434]:
wtayear_end_ranking2017['hand'].value_counts()

R    85
L    10
U     5
Name: hand, dtype: int64

In [435]:
wtayear_end_ranking2016['hand'].value_counts()

R    91
L     8
U     1
Name: hand, dtype: int64

In [436]:
wtayear_end_ranking2015['hand'].value_counts()

R    94
L     6
Name: hand, dtype: int64

In [437]:
wtayear_end_ranking2014['hand'].value_counts()

R    94
L     6
Name: hand, dtype: int64

In [438]:
wtayear_end_ranking2013['hand'].value_counts()

R    93
L     7
Name: hand, dtype: int64

In [439]:
wtayear_end_ranking2012['hand'].value_counts()

R    90
L    10
Name: hand, dtype: int64

In [440]:
wtayear_end_ranking2011['hand'].value_counts()

R    90
L     8
U     2
Name: hand, dtype: int64

In [441]:
wtayear_end_ranking2010['hand'].value_counts()

R    88
L    10
U     2
Name: hand, dtype: int64